In [7]:
# requirements:
! pip install huggingface_hub
! git config --global credential.helper store

In [8]:
from huggingface_hub import notebook_login
from huggingface_hub import HfFolder


#enter your API key, you can make one for free on HF
notebook_login()

Login successful
Your token has been saved to C:\Users\Markus/.huggingface/token


In [9]:
from huggingface_hub import InferenceApi

inference = InferenceApi("bigscience/bloom",token=HfFolder.get_token())

In [10]:
import time

def infer(prompt,
          max_length = 32,
          top_k = 0,
          num_beams = 0,
          no_repeat_ngram_size = 2,
          top_p = 0.9,
          seed=42,
          temperature=0.7,
          greedy_decoding = False,
          return_full_text = False):
    

    top_k = None if top_k == 0 else top_k
    do_sample = False if num_beams > 0 else not greedy_decoding
    num_beams = None if (greedy_decoding or num_beams == 0) else num_beams
    no_repeat_ngram_size = None if num_beams is None else no_repeat_ngram_size
    top_p = None if num_beams else top_p
    early_stopping = None if num_beams is None else num_beams > 0

    params = {
        "max_new_tokens": max_length,
        "top_k": top_k,
        "top_p": top_p,
        "temperature": temperature,
        "do_sample": do_sample,
        "seed": seed,
        "early_stopping":early_stopping,
        "no_repeat_ngram_size":no_repeat_ngram_size,
        "num_beams":num_beams,
        "return_full_text":return_full_text
    }
    
    s = time.time()
    response = inference(prompt, params=params)
    #print(response)
    proc_time = time.time()-s
    #print(f"Processing time was {proc_time} seconds")
    return response


In [219]:
def l(sentence, rationale, label):
    return """
    **Sentence:**
    """+sentence+"""
    
    **Rationale:**
    """+rationale+"""
    
    **Label:**
    """+label+"""
    
    """

def generate_prompt(sentence):
    prompt = """
    """+l("anyone consider maybe the missiles were oppressed us refugees looking for asylum in syria","","None")+"""
    """+l("you are a faggot sir","faggot","Hate")+"""
    """+l("...she says i will grab any woman by the pussy and fuck you brownies and fuck you fags and dykes us he a rapist racist homophobe transphobe you all how can you be so unpatriotic...","dykes","Hate")+"""
    """+l("now i hate the term libtard but people who are afraid of violence joker movie could incite are faggots","faggots","Hate")+"""
    """+l("another shit thread bumped up by the gab faggot","faggot","Hate")+"""
    """+l("go to bed faggot","faggot","Hate")+"""
    """+l("whoever made this thread has the big gay","gay","Hate")+"""
    """+l("faggots who put such an extreme emphasis on bourgouise education are like retarded children or maybe more like a normie republican who do not understand nature vs nurture wtf is wrong w you","faggots, retarded","Hate")+"""
    **Sentence:**
    """+sentence+"""

    **Rationale:**
    """
    return prompt

print(generate_prompt("##example_sentence##"))

def prompt_checkhate(sentence):

    prompt = generate_prompt(sentence)
    resp = infer(prompt)

    string = resp[0]['generated_text']
    answer = string.replace(prompt, "")
    answer = """
    **Sentence:**
    """+sentence+"""

    **Rationale:**
    """+answer
    
    # where is "Label"
    start_index = answer.find("Label")
    if(start_index == -1):
        print("ERROR: Prompt engine cannot interfere")
        return
    # cut one \n after   
    cut = len(answer)
    once_more = False
    for i in range (len(answer)-1):
        index = answer.find("\n", i, i+1)
        if(index > start_index):
            if(once_more):
                cut = index
                #print("cut is "+str(cut)+", start_index is "+str(start_index))
                break
                
            once_more = True
            continue

    answer = answer[0:cut]
    print(answer)


    
    **Sentence:**
    anyone consider maybe the missiles were oppressed us refugees looking for asylum in syria
    
    **Rationale:**
    
    
    **Label:**
    None
    
    
    
    **Sentence:**
    you are a faggot sir
    
    **Rationale:**
    faggot
    
    **Label:**
    Hate
    
    
    
    **Sentence:**
    ...she says i will grab any woman by the pussy and fuck you brownies and fuck you fags and dykes us he a rapist racist homophobe transphobe you all how can you be so unpatriotic...
    
    **Rationale:**
    dykes
    
    **Label:**
    Hate
    
    
    
    **Sentence:**
    now i hate the term libtard but people who are afraid of violence joker movie could incite are faggots
    
    **Rationale:**
    faggots
    
    **Label:**
    Hate
    
    
    
    **Sentence:**
    another shit thread bumped up by the gab faggot
    
    **Rationale:**
    faggot
    
    **Label:**
    Hate
    
    
    
    **Sentence:**
    go to bed faggot
    
    **Ra

In [220]:
prompt_checkhate("take yo bitch ass back to the east")


    **Sentence:**
    take yo bitch ass back to the east

    **Rationale:**
    
    
    **Label:**
    Hate


In [221]:
prompt_checkhate("I love you you are so nice to me")


    **Sentence:**
    I love you you are so nice to me

    **Rationale:**
    
    
    **Label:**
    None


In [222]:
prompt_checkhate("you are a faggot sir")

ERROR: Prompt engine cannot interfere


In [223]:
prompt_checkhate("this is what you nazi faggots do all day")


    **Sentence:**
    this is what you nazi faggots do all day

    **Rationale:**
    
    
    **Label:**
    Hate
